In [ ]:

import numpy as np
import pygame, sys
from pygame.locals import *
import random

pygame.init()

# Constants, Classes, Function definitions at top
# Code logic runs at bottom

# TODO:
#   1. give resetBoard's current logic to newBoard(). resetBoard() currently 
#      generates a new game (change logic to have it play same game) 
#   2. add timer to display for game
#   3. add a winning message/prompt before starting a new game after winning
#   4. have quit end program gracefully (currently needs to be force quit)

# Color codes
WHITE = (255,255,255)
RED = (255,0,0)
GREEN = (0, 255, 0)
PURPLE =(102, 0, 204)
YELLOW = (255,255,0)
BLUE = (0,0,255)
GRAY = (128,128,128)

### I started to use constants so we could change the size values buttttt
### we would have to adjust screen size and how we get coordinates etc.
### It might be too hard to do all of that

#define board size
BOARDSIZE = 5
#define square size
SQUARESIZE = 48
#define number of clicks
clickCounter = 0
#keep track of played boards
playedBoards = []
availableBoards = list(range(1,31))

### from https://inventwithpython.com/pygame/chapter2.html
### creates display surface
DISPLAYSURF = pygame.display.set_mode((300, 400))
###
pygame.display.set_caption('Lights Out') #window title
#Set font
font = pygame.font.Font('freesansbold.ttf', 15) 

        
class Square():
    def __init__(self, x, y, size, isOn, clickCount):
        self.size = size
        self.x = x
        self.y = y
        self.isOn = isOn
        self.clickCount = clickCount

    # Redraws square with new color
    def clicked(self):
        if self.isOn == True:
            pygame.draw.rect(DISPLAYSURF, PURPLE,\
                             (self.x,self.y, \
                              self.size, self.size))
            self.isOn = False
        elif self.isOn == False:
            pygame.draw.rect(DISPLAYSURF, WHITE,\
                             (self.x,self.y, \
                              self.size, self.size))
            self.isOn = True    
        
    def getx(self):
        return(self.x)
    
    def gety(self):
        return(self.y)
    
    def turnOn(self):
        self.isOn = True;
        pygame.draw.rect(DISPLAYSURF, WHITE,\
                             (self.x,self.y, \
                              self.size, self.size))
        return True

class Board():
    def __init__(self, boardNumber, target, startingBoard, solutionBoard):
        self.boardNumber = boardNumber
        self.target = target
        self.startingBoard = startingBoard
        self.solutionBoard = solutionBoard
    
    def getBoardNumber(self):
        return(self.boardNumber)

    def getTarget(self):
        return(self.target)

    def getStartingBoard(self):
        return(self.startingBoard)

    def getSolutionBoard(self):
        return(self.solutionBoard)

###############################################################################    
# Creates visuals for squares based off starting matrix for game.
# Purple for initial state of Off.
# White for initial state of On.
###############################################################################
def initSquares():
    global clickCounter
    clickCounter = 0
    #print("initializing board", gameBoard.getBoardNumber())
    for i in range(0,BOARDSIZE):
        for j in range(0,BOARDSIZE):
            #this doesn't scale well with BOARDSIZE haha
            #can fix later
            if (gameBoard.startingBoard[i][j] == 0):
                squares[i][j] = Square(25+(i*50),25+(j*50),SQUARESIZE, False, 0)
                pygame.draw.rect(DISPLAYSURF, PURPLE, \
                                 (squares[i][j].x, squares[i][j].y, \
                                  squares[i][j].size, squares[i][j].size))

            else:
                squares[i][j] = Square(25+(i*50),25+(j*50),SQUARESIZE, True, 0)
                pygame.draw.rect(DISPLAYSURF, WHITE, \
                                 (squares[i][j].x, squares[i][j].y, \
                                  squares[i][j].size, squares[i][j].size))
                squares[i][j].turnOn()
                
def initButtons():
    #Set font 
    text = font.render("Moves: " + str(clickCounter), True,WHITE) 
    clicksRect = text.get_rect()   
    # set the center of the rectangular object. 
    clicksRect.center = (75,300)
    DISPLAYSURF.blit(text, clicksRect)
    
    targetText = font.render("Target: " + str(gameBoard.target), True,WHITE) 
    targetRect = targetText.get_rect()   
    targetRect.center = (225,300)
    DISPLAYSURF.blit(targetText, targetRect)
    
                                      #X,Y,WIDTH,HEIGHT
    pygame.draw.rect(DISPLAYSURF, RED,(25,350,100,25))
    pygame.draw.rect(DISPLAYSURF, GREEN,(25,315,100,25))
    pygame.draw.rect(DISPLAYSURF, GRAY,(175,315,100,25))
    pygame.draw.rect(DISPLAYSURF, BLUE,(175,350,100,25))

    
    text1 = font.render("Reset Board",True, WHITE)
    resetRect = text1.get_rect()   
    resetRect.center = ((175+(100/2)),(350+(25/2)))
    DISPLAYSURF.blit(text1, resetRect) 
    
    text2 = font.render("New Board",True, WHITE)
    NBRect = text2.get_rect()   
    NBRect.center = ((25+(100/2)),(315+(25/2)))
    DISPLAYSURF.blit(text2, NBRect) 
    
    text3 = font.render("Hint",True, WHITE)
    hintRect = text3.get_rect()   
    hintRect.center = ((175+(100/2)),(315+(25/2)))
    DISPLAYSURF.blit(text3, hintRect) 
    
    text4 = font.render("Quit",True, WHITE)
    quitRect = text4.get_rect()   
    quitRect.center = ((25+(100/2)),(350+(25/2)))
    DISPLAYSURF.blit(text4, quitRect) 
    
def click():
    global clickCounter
    clickCounter += 1
    text = font.render("Moves: "+str(clickCounter), True,WHITE) 
    textRect = text.get_rect()   
    # set the center of the rectangular object. 
    textRect.center = (75,300)
    #clears the old text
    DISPLAYSURF.fill(pygame.Color("black"), (0, 290, 300, 20))
    #display new text
    DISPLAYSURF.blit(text, textRect)
    
    targetText = font.render("Target: " + str(gameBoard.target), True,WHITE) 
    targetRect = targetText.get_rect()   
    targetRect.center = (225,300)
    DISPLAYSURF.blit(targetText, targetRect)

def clicked_square():
    pos = pygame.mouse.get_pos() #where mouse clicked
    x = pos[0]
    y = pos[1] 
    
    if ((x<=125 and x>=25) and (y>=350 and y<=375)):
        quitGame()
    
    elif ((x<=275 and x>=175) and (y>=315 and y<=340)):
        hint(gameBoard)
    
    elif ((x<=275 and x>=175) and (y>=350 and y<=375)):
        resetBoard()
    
    elif ((x<=125 and x>=25) and (y>=315 and y<=340)):
        newBoard()
    
    #None would be outside game board
    elif((x<=275 and x>=25) and (y>=25 and y<=275)):
        if(getSquare(x,y) is None):
            return
        xi,yj = getSquare(x,y)#find out which square was clicked
        squares[xi][yj].clicked() #click the square
        squares[xi][yj].clickCount = squares[xi][yj].clickCount + 1
        
        """changing the squares around the one that was clicked"""
        if(xi > 0): squares[xi-1][yj].clicked()
        if(yj > 0): squares[xi][yj-1].clicked()
        if(xi < 4): squares[xi+1][yj].clicked()
        if(yj < 4): squares[xi][yj+1].clicked()
    
        click()
        checkWin(); #check if this was the winning move

###############################################################################
# Picks a board set from BoardSets.txt:
#    Description of board scheme breakdown takes first 6 lines 
#    First board starts on line 7
#    Every board set is 5 lines, second starts on 12, third starts on 17..
#       randomly generate X, a number (0, number of boards - 1)
#       multiply X by 5 (accounts for each set of board data)
#       add 6 (accounts for instructions on first 6 lines)
###############################################################################
def selectBoard():    
    global availableBoards
    #n = random.randint(0, 29)
    n = random.choice(availableBoards)-1
    boardSetStart = (n * 5) + 6

    boardNumber = int(boardList[boardSetStart])
    target = int(boardList[boardSetStart + 1])
    rawStartingBoard = boardList[boardSetStart + 2]
    rawSolutionBoard =boardList[boardSetStart + 3]

    startingBoard = cleanBoardInput(rawStartingBoard)
    solutionBoard = cleanBoardInput(rawSolutionBoard)

    board = Board(boardNumber, target, startingBoard, solutionBoard)

    return(board)
###############################################################################
# Converts board input from string to output as 5 x 5 matrix
###############################################################################
def cleanBoardInput(boardStr):
    boardStr = boardStr.replace("[","")
    boardStr = boardStr.replace("]","")
    boardStr = boardStr.replace(",","")

    boardContents = boardStr.split()

    for i in range(0, len(boardContents)): 
        boardContents[i] = int(boardContents[i])

    startingMatrix = np.array(boardContents)
    startingMatrix = startingMatrix.reshape(5, 5) 

    return (startingMatrix)

def getSquare(x,y):
    # determine which square you're in by comparing coordinates to 
    for i in range(0, BOARDSIZE):
        for j in range(0, BOARDSIZE):
            if((x > squares[i][j].getx()) & (x < squares[i][j].getx()+45)\
            &(y > squares[i][j].gety()) & (y < squares[i][j].gety()+45)):
                return(i,j)
    # not in a square
    return; 

def quitGame():
    pygame.display.quit()
    pygame.quit()
    sys.exit() 

###############################################################################
# Logic for hint button:
# In order to win, each square must be clicked an even number of times
# (effectively equivalent to having never been clicked), or an odd number 
# of times (effectively equivalent to having been clicked once).
# Finds a square that must be clicked once more. Displays that square in yellow.
###############################################################################
def hint(gameBoard):
    xi = -1
    yj = -1
    
    for i in range(0, BOARDSIZE):
        for j in range(0, BOARDSIZE):
            if((squares[i][j].clickCount % 2) != gameBoard.solutionBoard[i][j]):
                xi = i
                yj = j
                
    squares[xi][yj] = Square(25+(xi*50),25+(yj*50),SQUARESIZE, \
                             squares[xi][yj].isOn, squares[xi][yj].clickCount)
    
    pygame.draw.rect(DISPLAYSURF, YELLOW, \
                (squares[xi][yj].x, squares[xi][yj].y, \
                squares[xi][yj].size, squares[xi][yj].size))

# TODO: 
# Add timer to display current time spent on board
def timer():
    print("Add timer to display current time spent on board")

# 
# Generates a new game board from a list of available boards
# Once a game board has been played, it is removed from the list
# When all boards have been played, the list resets 
#
def newBoard():
    global gameBoard
    global availableBoards
    global playedBoards
    global clickCounter

    #print("current board:", gameBoard.getBoardNumber())
    
    while(availableBoards):
        gameBoard = selectBoard()
        if(gameBoard.getBoardNumber() in availableBoards):
            initSquares()
            #print("boards played: ", playedBoards, "boards available", availableBoards)
            playedBoards.append(gameBoard.getBoardNumber())
            availableBoards.remove(gameBoard.getBoardNumber())
            
            text = font.render("Moves: "+str(clickCounter), True,WHITE) 
            textRect = text.get_rect()   
            # set the center of the rectangular object. 
            textRect.center = (75,300)
            #clears the old text
            DISPLAYSURF.fill(pygame.Color("black"), (0, 290, 300, 20))
            #display new text
            DISPLAYSURF.blit(text, textRect)
    
            targetText = font.render("Target: " + str(gameBoard.target), True,WHITE) 
            targetRect = targetText.get_rect()   
            targetRect.center = (225,300)
            DISPLAYSURF.blit(targetText, targetRect)
            return
    
    #print(playedBoards)
    #print(set(playedBoards))
    #reset board lists
    availableBoards = list(range(1,31))
    playedBoards = []
    
# TODO:
# change resetBoard() logic to have it play same board
def resetBoard():
    initSquares()
    gameBoard = selectBoard()
    global clickCounter
    text = font.render("Moves: "+str(clickCounter), True,WHITE) 
    textRect = text.get_rect()   
    # set the center of the rectangular object. 
    textRect.center = (75,300)
    #clears the old text
    DISPLAYSURF.fill(pygame.Color("black"), (0, 290, 300, 20))
    #display new text
    DISPLAYSURF.blit(text, textRect)
    
    targetText = font.render("Target: " + str(gameBoard.target), True,WHITE) 
    targetRect = targetText.get_rect()   
    targetRect.center = (225,300)
    DISPLAYSURF.blit(targetText, targetRect)

# Check if game is won
def checkWin():
    for i in range(0, BOARDSIZE):
        for j in range(0, BOARDSIZE):
            if(squares[i][j].isOn):
                return;
            elif(i==j==4):
                # TODO: create win function to do something
                print("win")
                initSquares()
                gameBoard = selectBoard()
    
                
"""   start here   """
# Load available game boards from BoardSets.txt
boardFile = "BoardSets.txt"
boardList = open(boardFile, 'r').readlines()
gameBoard = selectBoard()
print("initial board:", gameBoard.getBoardNumber())
#Keep track of which boards have been played
playedBoards.append(gameBoard.getBoardNumber())
availableBoards.remove(gameBoard.getBoardNumber())


# Array to hold squares
squares = np.ndarray(shape=(BOARDSIZE,BOARDSIZE), dtype=Rect, order='F')                

# Create game board
initSquares()

# Create buttons
initButtons()

# Main game loop
while True:
    
    for event in pygame.event.get():
        if event.type == pygame.MOUSEBUTTONDOWN:
            clicked_square()
            
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
    pygame.display.update()


## 